In [19]:
import pandas as pd

In [20]:
grid = pd.DataFrame(columns = ["lat", "long", "population"])

GET GRIDS

In [24]:
import sys
sys.path.append("../tools")
# from grids import pxtolat, pxtolong

In [39]:
def pxtolong(px, width):
    adjlong = (px / width) * 360 
    return adjlong - 180

def pxtolat(px, height):
    adjlat = (px/height) * 180
    return -(adjlat -90)

In [63]:
from tqdm import tqdm

d = {
    'lat': [],
    'long': [],
    'population': []
}


with open('./gpw_pop.asc', 'r') as f:
        for i in range(6):
            f.readline() # skip pre data
        width = 8640
        height = 4320
        for rn in tqdm(range(0, 4320)):
            k = f.readline().split(' ')
            for c in range(0, 8640):
                cell_val = float(k[c]) # read population in grid

                if cell_val <= 0: # skip pixel if no population
                    continue
                # print(cell_val)
                d['lat'].append(pxtolat(rn, height))
                d['long'].append(pxtolong(c, width))
                d['population'].append(cell_val)
        
        
        grid = pd.DataFrame(d)

100%|██████████| 4320/4320 [00:27<00:00, 159.58it/s]


POPULATION

In [65]:
tmp = grid.loc[grid['population'] > 500000]
tmp.head(10)

,lat,long,population
2742573,45.791667,126.666667,621797.9
2926015,43.916667,125.250000,559883.8
2932979,43.833333,87.583333,520489.2
3108488,41.833333,123.333333,545899.1
3108490,41.833333,123.416667,601812.9
3168988,41.083333,28.833333,629407.6
3168989,41.083333,28.875000,543774.2
3172486,41.041667,28.791667,646545.8
3172487,41.041667,28.833333,818725.8
3172488,41.041667,28.875000,506676.8


EARTHQUAKES

TSUNAMI

HURRICANES

WIND

SOLAR